# SVM Emotion Recognition — Training Notebook
This notebook loads a CK-style CSV dataset, trains the SVM model, evaluates it, and saves both the model and scaler.

In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import joblib

try:
    from imblearn.over_sampling import SMOTE
    has_smote = True
except:
    has_smote = False

TARGET_SIZE = (48, 48)

In [9]:
def load_dataset(csv_path):
    df = pd.read_csv(csv_path)
    pixels = df['pixels'].apply(lambda s: np.fromstring(s, sep=' ', dtype=int))
    X = np.stack(pixels.values).astype('float32') / 255.0
    y = df['emotion'].astype(int).values
    return X, y

In [10]:
CSV_PATH = 'ckextended.csv'  # update your dataset path here
MODEL_PATH = 'svm_emotion_model.joblib'
SCALER_PATH = 'scaler.joblib'

X, y = load_dataset(CSV_PATH)
print('Dataset loaded:', X.shape, y.shape)

Dataset loaded: (920, 2304) (920,)


In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

if has_smote:
    sm = SMOTE(random_state=42)
    X_train_scaled, y_train = sm.fit_resample(X_train_scaled, y_train)
    print('Applied SMOTE.')

Applied SMOTE.


In [12]:
svm = SVC(kernel='rbf', probability=True, class_weight='balanced', random_state=42)
svm.fit(X_train_scaled, y_train)

y_pred = svm.predict(X_test_scaled)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.50      0.11      0.18         9
           1       0.88      0.58      0.70        12
           2       0.00      0.00      0.00         5
           3       0.82      0.64      0.72        14
           4       0.00      0.00      0.00         5
           5       1.00      0.69      0.81        16
           6       0.79      0.97      0.87       119
           7       0.20      0.25      0.22         4

    accuracy                           0.78       184
   macro avg       0.52      0.41      0.44       184
weighted avg       0.75      0.78      0.75       184



c:\Users\Aaron\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\Aaron\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\Aaron\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(ave

In [13]:
joblib.dump(svm, MODEL_PATH)
joblib.dump(scaler, SCALER_PATH)
print('Model and scaler saved.')

Model and scaler saved.
